In [1]:
print("Ok")

Ok


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [3]:
PINECONE_API_KEY = "pcsk_YCVy2_MkRjuLP39ThgW9q273iyt9Jx6kTaLNzkHapSeGQUoXB4JRsDMDJunpu9ykzJSxd"
PINECONE_API_ENV = "us-east-1-aws"

In [ ]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents